In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
n_users = 185
gpu_id = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
# Download the dataset
download_elec_dataset()

# Clean dataset
elec_data = clean_elec()

from sklearn.preprocessing import StandardScaler, MinMaxScaler
Std_normalization = 1
if Std_normalization:
    scaler = StandardScaler()
    temp = scaler.fit_transform(elec_data)
    norm_means = scaler.mean_
    norm_std = scaler.scale_
else:
    scaler = MinMaxScaler()
    temp = scaler.fit_transform(elec_data)

elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

Files are ready
Data loaded..
Data aggregated by hour: (26304, 370)


In [6]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# elec_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/electricity/electricity.csv', index_label='date')


In [7]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(elec_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.data_path

'electricity.csv'

In [8]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [9]:
test_data[0][0].shape

(96, 371)

In [10]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [11]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
            

# Setup WANDB

In [12]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Elec dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Elec dataset",
    # Hyperparams
    config={
        "dataset": "Elec370",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 370,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [13]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 45.92900896072388
Epoch: 2 cost time: 43.63919711112976
Epoch: 3 cost time: 44.52861046791077
Epoch: 4 cost time: 43.51661944389343
Epoch: 5 cost time: 48.52350425720215
Epoch: 1 cost time: 55.41747307777405
Epoch: 2 cost time: 48.77126407623291
Epoch: 3 cost time: 44.38904166221619
Epoch: 4 cost time: 44.67761516571045
Epoch: 5 cost time: 45.20271348953247
Epoch: 1 cost time: 43.980788230895996
Epoch: 2 cost time: 44.440834760665894
Epoch: 3 cost time: 44.7698609828949
Epoch: 4 cost time: 43.913169622421265
Epoch: 5 cost time: 44.16525983810425
Epoch: 1 cost time: 43.82369923591614
Epoch: 2 cost time: 43.973313331604004
Epoch: 3 cost time: 44.487173080444336
Epoch: 4 cost time: 45.03045892715454
Epoch: 5 cost time: 46.48386192321777
Epoch: 1 cost time: 57.80879092216492
Epoch: 2 cost time: 120.5734372138977
Epoch: 3 cost time: 161.46365928649902
Epoch: 4 cost time: 228.55656099319458
Epoch: 5 cost time: 305.55224800109863
Epoch: 1 cost time: 327.2169861793518
Epoch

Progress:   3%|▎         | 1/30 [7:00:21<203:10:23, 25221.51s/it]

Epoch: 1 cost time: 333.8003077507019
Epoch: 2 cost time: 333.7177984714508
Epoch: 3 cost time: 335.67944502830505
Epoch: 4 cost time: 335.6864182949066
Epoch: 5 cost time: 333.5435290336609
Epoch: 1 cost time: 335.5144724845886
Epoch: 2 cost time: 334.53327918052673
Epoch: 3 cost time: 335.84737849235535
Epoch: 4 cost time: 333.71725249290466
Epoch: 5 cost time: 336.1158356666565
Epoch: 1 cost time: 319.9102985858917
Epoch: 2 cost time: 303.6918828487396
Epoch: 3 cost time: 304.5052444934845
Epoch: 4 cost time: 303.81628799438477
Epoch: 5 cost time: 306.2849519252777
Epoch: 1 cost time: 303.77718448638916
Epoch: 2 cost time: 303.36447525024414
Epoch: 3 cost time: 305.747465133667
Epoch: 4 cost time: 303.13874888420105
Epoch: 5 cost time: 305.5692641735077
Epoch: 1 cost time: 306.0720410346985
Epoch: 2 cost time: 306.45097064971924
Epoch: 3 cost time: 304.1257812976837
Epoch: 4 cost time: 302.3465995788574
Epoch: 5 cost time: 304.9236295223236
Epoch: 1 cost time: 304.51397156715393
Epo

Progress:   7%|▋         | 2/30 [14:28:53<203:54:11, 26216.12s/it]

...

Epoch: 1 cost time: 43.98151683807373
Epoch: 2 cost time: 44.403234004974365
Epoch: 3 cost time: 44.52331852912903
Epoch: 4 cost time: 44.05326247215271
Epoch: 5 cost time: 43.830010414123535
Epoch: 1 cost time: 44.140368700027466
Epoch: 2 cost time: 43.934884786605835
Epoch: 3 cost time: 43.54831647872925
Epoch: 4 cost time: 44.00043964385986
Epoch: 5 cost time: 43.818411350250244
Epoch: 1 cost time: 43.381415367126465
Epoch: 2 cost time: 44.56982946395874
Epoch: 3 cost time: 43.489996671676636
Epoch: 4 cost time: 43.942564964294434
Epoch: 5 cost time: 43.863654375076294
Epoch: 1 cost time: 43.26590180397034
Epoch: 2 cost time: 43.59467434883118
Epoch: 3 cost time: 43.11353063583374
Epoch: 4 cost time: 43.734896183013916
Epoch: 5 cost time: 44.20631551742554
Epoch: 1 cost time: 44.7569215297699
Epoch: 2 cost time: 43.13982272148132
Epoch: 3 cost time: 44.38584327697754
Epoch: 4 cost time: 44.65027332305908
Epoch: 5 cost time: 43.65071105957031
Epoch: 1 cost time: 43.60628581047058
Epo

Progress:  57%|█████▋    | 17/30 [49:27:48<15:54:01, 4403.17s/it]

Epoch: 1 cost time: 38.63982272148132
Epoch: 2 cost time: 38.90498471260071
Epoch: 3 cost time: 38.759735107421875
Epoch: 4 cost time: 41.63122797012329
Epoch: 5 cost time: 43.471559047698975
Epoch: 1 cost time: 44.83483076095581
Epoch: 2 cost time: 44.7510621547699
Epoch: 3 cost time: 44.07858157157898
Epoch: 4 cost time: 44.45230317115784
Epoch: 5 cost time: 44.63082218170166
Epoch: 1 cost time: 43.678791761398315
Epoch: 2 cost time: 44.054436683654785
Epoch: 3 cost time: 44.01458430290222
Epoch: 4 cost time: 44.07881236076355
Epoch: 5 cost time: 44.34410572052002
Epoch: 1 cost time: 44.08920407295227
Epoch: 2 cost time: 44.39237976074219
Epoch: 3 cost time: 44.560930013656616
Epoch: 4 cost time: 45.01073217391968
Epoch: 5 cost time: 44.78090977668762
Epoch: 1 cost time: 44.263890981674194
Epoch: 2 cost time: 43.8115177154541
Epoch: 3 cost time: 43.09795022010803
Epoch: 4 cost time: 44.14646935462952
Epoch: 5 cost time: 43.65461206436157
Epoch: 1 cost time: 44.072447538375854
Epoch: 

Progress:  60%|██████    | 18/30 [50:35:49<14:21:18, 4306.55s/it]

Epoch: 1 cost time: 44.01851511001587
Epoch: 2 cost time: 44.78701162338257
Epoch: 3 cost time: 44.08750009536743
Epoch: 4 cost time: 43.63183331489563
Epoch: 5 cost time: 43.8295636177063
Epoch: 1 cost time: 43.817848205566406
Epoch: 2 cost time: 44.26588821411133
Epoch: 3 cost time: 43.73108196258545
Epoch: 4 cost time: 44.434921979904175
Epoch: 5 cost time: 43.983962297439575
Epoch: 1 cost time: 44.094539165496826
Epoch: 2 cost time: 43.88835906982422
Epoch: 3 cost time: 43.758243799209595
Epoch: 4 cost time: 44.599987745285034
Epoch: 5 cost time: 44.21602654457092
Epoch: 1 cost time: 43.78221583366394
Epoch: 2 cost time: 44.31162238121033
Epoch: 3 cost time: 43.99243092536926
Epoch: 4 cost time: 44.07529139518738
Epoch: 5 cost time: 43.60109210014343
Epoch: 1 cost time: 44.027750968933105
Epoch: 2 cost time: 43.78390693664551
Epoch: 3 cost time: 43.130910873413086
Epoch: 4 cost time: 43.623064041137695
Epoch: 5 cost time: 43.8281044960022
Epoch: 1 cost time: 43.83427572250366
Epoch